In [30]:
import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import anansescanpy as asc
import os
import numpy as np

work_dir = '../../output'
outputdir=f"{work_dir}/ananse/"
# os.makedirs(outputdir)

# Fill in the directories where the h5ad RNA and ATAC objects are located
atac_PBMC=sc.read(f'{work_dir}/scATAC/adata_atac.h5ad')
rna_PBMC=sc.read(f'{work_dir}/scRNA/adata_rna.h5ad')

def filter_chr(peaks):
    chrs = [peak.split(':')[0] for peak in peaks]
    mask = np.asarray([True if ('chr' in chr) else False for chr in chrs])
    return mask
    
mask = filter_chr(atac_PBMC.var.index)

atac_PBMC = atac_PBMC[:, mask]

rna_PBMC.obs['cell_type']=rna_PBMC.obs['cell_type'].str.replace(' ', '-').astype('category')
rna_PBMC.obs['cell_type']=rna_PBMC.obs['cell_type'].str.replace('_', '-').astype('category')
atac_PBMC.obs['cell_type']=atac_PBMC.obs['cell_type'].str.replace(' ', '-').astype('category')
atac_PBMC.obs['cell_type']=atac_PBMC.obs['cell_type'].str.replace('_', '-').astype('category')


minimal=25
asc.export_CPM_scANANSE(anndata=rna_PBMC,
min_cells=minimal,
outputdir=outputdir,
cluster_id="cell_type"
)
asc.export_ATAC_scANANSE(anndata=atac_PBMC,
min_cells=minimal,
outputdir=outputdir,
cluster_id="cell_type"
)
asc.config_scANANSE(anndata=rna_PBMC,
min_cells=minimal,
outputdir=outputdir,
cluster_id="cell_type"
)
asc.DEGS_scANANSE(anndata=rna_PBMC,
min_cells=minimal,
outputdir=outputdir,
cluster_id="cell_type"
)

AnnData object with n_obs × n_vars = 25551 × 135418
    obs: 'obs_id', 'cell_type', 'donor_id'
    uns: 'cell_type_colors', 'donor_id_colors', 'neighbors', 'obs', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    layers: 'x_norm'
    obsp: 'connectivities', 'distances'

gather data from B-cells with 3809 cells
gather data from agg-type with 10371 cells
gather data from NK-cells with 4497 cells
gather data from Myeloid-cells with 6874 cells
gather data from B-cells with 3809 cells
gather data from agg-type with 10371 cells
gather data from NK-cells with 4497 cells
gather data from Myeloid-cells with 6874 cells
anansesnake_B-cells_average
anansesnake_agg-type_average
anansesnake_NK-cells_average
anansesnake_Myeloid-cells_average
calculating DEGS for contrast anansesnake_B-cells_average
skip
calculating DEGS for contrast anansesnake_agg-type_average
calculating DEGS for contrast anansesnake_NK-cells_average
calculating DEGS for contrast anansesnake_Myeloid-cells_average
